In [1]:
import numpy as np
import argparse
import time
import os
import collections
import json
import pickle
from scipy import special
import scipy.spatial.distance
from scipy.optimize import linear_sum_assignment

from utils.data_utils import load_dataset_numpy

In [2]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Function definitions

In [3]:
def acc_fn(Y,Y_pred):
    count_correct = 0.0
    for i in range(len(Y)):
        if Y_pred[i] == Y[i]:
            count_correct += 1
    acc = count_correct/len(Y)
    return acc

In [4]:
def empirical_cost(X_1,X_2,eps):
    D_12 = scipy.spatial.distance.cdist(X_1,X_2,metric='euclidean')
#     print(np.mean(D_12))
    cost_matrix = D_12 > 2*eps
    cost_matrix = cost_matrix.astype(float)
    output = linear_sum_assignment(cost_matrix)
    costs = cost_matrix[output[0], output[1]]
    cost_zero_indices = np.where(costs==0.0)
    matching_indices = (output[0][cost_zero_indices], output[1][cost_zero_indices])
    raw_cost = np.float(cost_matrix[output[0], output[1]].sum())
    mean_cost = raw_cost/(num_samples)
    min_error = (1-mean_cost)/2
    return min_error, mean_cost

## Defining Gaussian model

In [5]:
alpha_star=special.erfinv(0.98)*np.sqrt(2)

In [6]:
alpha_star

2.3263478740408408

### Parameters

In [ ]:
d=10

In [ ]:
evalues=np.random.default_rng().uniform(0,1,d)

In [ ]:
sigma = np.diag(evalues)
sigma_inv = np.linalg.inv(sigma)

In [ ]:
mu=(alpha_star)*np.sqrt(evalues)/np.sqrt(d)

In [ ]:
np.dot(mu,np.dot(sigma_inv,mu))

In [ ]:
mean_dist=np.linalg.norm(2*mu)
mean_dist

### Optimal classifier loss for L2 norm

In [ ]:
delta_init = np.linspace(0,10,19)
rhs_vec=mu
w_opt=[]
eps_opt=[]
for delta in delta_init:
    lhs_mat = 0.5*(sigma+2*delta*np.eye(d))
    w=np.linalg.solve(lhs_mat,rhs_vec)
    eps_curr=delta*np.linalg.norm(w)
    print('Optimal for eps %s' % eps_curr)
    w_opt.append(w)
    eps_opt.append(eps_curr)

In [ ]:
optimal_losses = []
for i in range(len(delta_init)):
#     alpha = np.sqrt((np.dot(w_opt[i],mu)-eps_opt[i]*np.linalg.norm(w_opt[i]))/2)
    alpha = (-eps_opt[i]*np.linalg.norm(w_opt[i])+np.dot(w_opt[i],mu))/np.sqrt(np.dot(w_opt[i],np.dot(sigma,w_opt[i])))
    Q_alpha = 0.5 - 0.5*special.erf(alpha/np.sqrt(2))
    optimal_loss=Q_alpha
    optimal_losses.append(optimal_loss)
    print('Optimal loss at %s is %s' % (eps_opt[i],optimal_loss))

## Sampling

### Drawing samples from Gaussian

In [ ]:
rng = np.random.default_rng(77)

In [ ]:
num_samples=500000
X_1 = rng.multivariate_normal(-1.0*mu, sigma, (num_samples))
X_2 = rng.multivariate_normal(mu, sigma, (num_samples))
X=np.vstack((X_1,X_2))

In [ ]:
Y=np.zeros(2*num_samples)
Y[num_samples:]=1

In [ ]:
det=np.linalg.det(sigma)
den=((2*3.14)**d)*det
prob=(1/den)*np.exp(-0.5*np.dot(X_2[100]-mu,np.dot(sigma_inv,X_2[100]-mu)))

In [ ]:
prob

In [ ]:
num_samples_test=500
X_1_test = rng.multivariate_normal(-1.0*mu, sigma, (num_samples))
X_2_test = rng.multivariate_normal(mu, sigma, (num_samples))
X_test=np.vstack((X_1_test,X_2_test))

In [ ]:
Y_test=np.zeros(2*num_samples_test)
Y_test[num_samples_test:]=1

### Benign classification

In [ ]:
def predict_linear(w,X):
    sum_of_means = 2*mu
    b=0.5*np.dot(np.dot(sum_of_means,sigma_inv),mu)
#     b=0.5*(np.dot(clf.means_[0],np.dot(sigma_inv,clf.means_[0]))-np.dot(clf.means_[1],np.dot(sigma_inv,clf.means_[1])))
    # sign reversal
    score = np.dot(X,w) - b
    prediction = np.sign(score)
    prediction[prediction==-1] = 0
    return prediction

In [ ]:
w_lda = 2*np.dot(sigma_inv,mu)
Y_pred_lda_extract = predict_linear(w_lda, X)
acc_ben_lda_extract = acc_fn(Y,Y_pred_lda_extract)
acc_ben_lda_extract

In [ ]:
Y_pred_trial = predict_linear(w_opt[0], X)
acc_ben_trial = acc_fn(Y,Y_pred_trial)
acc_ben_trial

### Computing optimal empirical matching

In [ ]:
# Run bipartite matching here
empirical_loss_min, cost = empirical_cost(X_1,X_2,3.295441890935306)

In [ ]:
empirical_loss_min

In [ ]:
empirical_loss_min*num_samples*2

# Loop

In [26]:
rng = np.random.default_rng(77)
d_list = [2,10,50,100]
# d_list=[10]
param_list = []
sample_list = []
sample_list_test = []
emp_cost_list = []
eps_opt_list = []
optimal_loss_list = []
w_opt_list=[]
for i,d in enumerate(d_list):
    print(i,d)
    # Gaussian params
    evalues=np.random.default_rng().uniform(0,1,d)
    sigma = np.diag(evalues)
    sigma_inv = np.linalg.inv(sigma)
    mu=(alpha_star)*np.sqrt(evalues)/np.sqrt(d)
    param_list.append([mu,evalues])
    # Run eps search
    delta_init = np.linspace(0,10,19)
    rhs_vec=mu
    w_opt=[]
    eps_opt=[]
    for delta in delta_init:
        lhs_mat = 0.5*(sigma+2*delta*np.eye(d))
        w=np.linalg.solve(lhs_mat,rhs_vec)
        eps_curr=delta*np.linalg.norm(w)
        #print('Optimal for eps %s' % eps_curr)
        w_opt.append(w)
        eps_opt.append(eps_curr)
    eps_opt_list.append(eps_opt)
    w_opt_list.append(w_opt)
    # Compute optimal loss
    optimal_losses = []
    for i in range(len(delta_init)):
#         alpha = np.sqrt((np.dot(w_opt[i],mu)-eps_opt[i]*np.linalg.norm(w_opt[i]))/2)
        alpha = (-eps_opt[i]*np.linalg.norm(w_opt[i])+np.dot(w_opt[i],mu))/np.sqrt(np.dot(w_opt[i],np.dot(sigma,w_opt[i])))
        Q_alpha = 0.5 - 0.5*special.erf(alpha/np.sqrt(2))
        optimal_loss=Q_alpha
        optimal_losses.append(optimal_loss)
        #print('Optimal loss at %s is %s' % (eps_opt[i],optimal_loss))
    optimal_loss_list.append(optimal_losses)
    # Generate samples
    sample_num_list = [500,1000,5000]
#     sample_num_list = [5000]
    samples=[]
    emp_costs=[]
    for j, num_samples in enumerate(sample_num_list):
        print(j,num_samples)
        X_1 = rng.multivariate_normal(-1.0*mu, sigma, (num_samples))
        X_2 = rng.multivariate_normal(mu, sigma, (num_samples))
        X=np.vstack((X_1,X_2))
        samples.append(X)
        emp_costs_per_sample=[]
        for eps in eps_opt:
            empirical_loss_min, cost = empirical_cost(X_1,X_2,eps)
            emp_costs_per_sample.append(empirical_loss_min)
        emp_costs.append(emp_costs_per_sample)
    emp_cost_list.append(emp_costs)
    sample_list.append(samples)
    # Generate test samples
    num_samples_test=5000
    X_1_test = rng.multivariate_normal(-1.0*mu, sigma, (num_samples_test))
    X_2_test = rng.multivariate_normal(mu, sigma, (num_samples_test))
    X_test=np.vstack((X_1_test,X_2_test))
    sample_list_test.append(X_test)
    print(optimal_losses)
    print(emp_costs)
PIK1 = "gauss_sample.dat"
with open(PIK1, "wb") as f:
    pickle.dump(sample_list, f)
PIK2 = "gauss_params.dat"
with open(PIK2, "wb") as f:
    pickle.dump(param_list, f)
PIK3 = "eps_opt.dat"
with open(PIK3, "wb") as f:
    pickle.dump(eps_opt_list, f)
PIK4 = "emp_opt_losses.dat"
with open(PIK4, "wb") as f:
    pickle.dump([optimal_loss_list,emp_cost_list], f)
# # PIK2 = "gauss_sample_test.dat"
# with open(PIK2, "wb") as f:
#     pickle.dump(sample_list_test, f)

0 2
0 500
1 1000
2 5000
[0.010000000000000009, 0.17153664159502874, 0.2757470177335013, 0.3321059980405139, 0.366350255827111, 0.38916066599253507, 0.40538752592825644, 0.417501153108054, 0.4268809795799948, 0.43435456429042574, 0.44044733043625134, 0.4455084541397765, 0.44977884159959924, 0.4534299762211535, 0.4565871951055494, 0.4593442060081497, 0.4617724572029466, 0.46392736744802326, 0.46585257710196104]
[[0.0, 0.15400000000000003, 0.245, 0.302, 0.33999999999999997, 0.361, 0.378, 0.39, 0.4, 0.41000000000000003, 0.415, 0.419, 0.424, 0.426, 0.431, 0.433, 0.434, 0.435, 0.438], [0.0, 0.16799999999999998, 0.26649999999999996, 0.323, 0.358, 0.3775, 0.3915, 0.40249999999999997, 0.4115, 0.4185, 0.4245, 0.4295, 0.434, 0.4385, 0.441, 0.4435, 0.4455, 0.449, 0.45], [0.0, 0.1643, 0.2693, 0.3281, 0.3639, 0.3875, 0.4033, 0.4158, 0.4249, 0.4327, 0.4387, 0.4442, 0.4494, 0.4535, 0.4563, 0.4589, 0.4612, 0.4629, 0.46499999999999997]]
1 10
0 500
1 1000
2 5000
[0.010000000000000009, 0.24859720138213065

### Empirical loss of robust classifier

In [29]:
import os 
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = ''

import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
# from torch.utils.tensorboard import SummaryWriter

from utils.train_utils import train_one_epoch, robust_train_one_epoch, eps_scheduler
from utils.test_utils import test, robust_test, robust_test_hybrid

In [30]:
class gauss_data():
    def __init__(self,X,Y):
        self.data=X
        self.targets=Y
    def __getitem__(self, index):
        img, target = self.data[index], int(self.targets[index])

        return img, target, index, 0, 0
    def __len__(self):
        return len(self.data)

In [31]:
# Create data loader
def load_data_gauss_train(X,Y):
    train_data = gauss_data(torch.from_numpy(X).float(),torch.from_numpy(Y).float())

    loader_train = torch.utils.data.DataLoader(train_data, 
                                batch_size=50,
                                shuffle=True)

    return loader_train

In [32]:
def load_data_gauss_test(X_test,Y_test):
    test_data = gauss_data(torch.from_numpy(X_test).float(),torch.from_numpy(Y_test).float())

    loader_test = torch.utils.data.DataLoader(test_data, 
                                batch_size=50,
                                shuffle=False)
    return loader_test

In [33]:
class fcn_3l(nn.Module):
    def __init__(self, n_classes=2, input_dim=100):
        super(fcn_3l, self).__init__()
        self.fc1 = nn.Linear(input_dim, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200,n_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [34]:
torch.random.manual_seed(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [35]:
parser = argparse.ArgumentParser()

# Train args
parser.add_argument('--lr_schedule', type=str, default='linear0')
parser.add_argument('--loss_fn', type=str, default='CE')

# Attack args
parser.add_argument('--attack', type=str, default='PGD_l2')
parser.add_argument('--epsilon', type=float, default=3.0)
parser.add_argument('--attack_iter', type=int, default=10)
parser.add_argument('--gamma', type=float, default=1.0)
parser.add_argument('--targeted', dest='targeted', action='store_true')
parser.add_argument('--clip_min', type=float, default=0)
parser.add_argument('--clip_max', type=float, default=1.0)
parser.add_argument('--rand_init', dest='rand_init', action='store_true')
parser.add_argument('--eps_schedule', type=int, default=0)
parser.add_argument('--num_restarts', type=int, default=1)

_StoreAction(option_strings=['--num_restarts'], dest='num_restarts', nargs=None, const=None, default=1, type=<class 'int'>, choices=None, help=None, metavar=None)

In [36]:
# Run adversarial training here
eps_idx_list=[1,5]
emp_losses_all=[]
for k in range(2):
    emp_losses=[]
    for i,dim in enumerate(d_list):
        emp_loss=[]
        print(emp_loss)
        eps_idx_curr=eps_idx_list[k]
        eps=eps_opt_list[i][eps_idx_curr].astype(np.float32)
#         eps=eps_list[i][k]
        args = parser.parse_args("--attack=PGD_l2 --epsilon={} --attack_iter=10 --gamma=2.5".format(eps).split())
        attack_params = {'attack': args.attack, 'epsilon': args.epsilon, 
                         'attack_iter': args.attack_iter, 'eps_step': args.epsilon*args.gamma/args.attack_iter,
                         'targeted': args.targeted, 'clip_min': args.clip_min,
                         'clip_max': args.clip_max,'rand_init': args.rand_init, 
                         'num_restarts': args.num_restarts}
        for j,num_samples in enumerate(sample_num_list):
            model_dir_name = 'trained_models/Gaussian/gauss_fcn_dim' + str(dim) +'_ns' + str(num_samples) + '_eps' + str(eps)
            model_dir_name += '/'
            if not os.path.exists(model_dir_name):
                os.makedirs(model_dir_name)
            print(dim,num_samples)
            X = sample_list[i][j]
            Y=np.zeros(2*num_samples)
            Y[num_samples:]=1
            X_test = sample_list_test[i]
            Y_test=np.zeros(2*num_samples_test)
            Y_test[num_samples_test:]=1

            loader_train = load_data_gauss_train(X,Y)
            loader_test = load_data_gauss_test(X_test,Y_test)

            delta = args.epsilon*args.gamma/args.attack_iter
            args.track_hard=False

            net = fcn_3l(2,dim)

            net = net.cuda()

            criterion = nn.CrossEntropyLoss() 
            # criterion = nn.CrossEntropyLoss()

            optimizer = torch.optim.SGD(net.parameters(),
                                        lr=0.1,
                                        momentum=0.9,
                                        weight_decay=2e-4)

            if args.lr_schedule == 'cosine':
                scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                        T_max=50, eta_min=0, last_epoch=-1)
            elif args.lr_schedule == 'linear0':
                scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100,150,200], gamma=0.1)

            early_stop_counter = 0
            # early_stop_thresh = 0.05
            best_loss_adv = 100.0

            for epoch in range(0, 50):
                start_time = time.time()
                lr = optimizer.param_groups[0]['lr']
                print('Current learning rate: {}'.format(lr))
                curr_loss, ben_loss = robust_train_one_epoch(net,
                                            optimizer, loader_train, args, eps, delta, 
                                            epoch, '', verbose=False)
                print('time_taken for #{} epoch = {:.3f}'.format(epoch+1, time.time()-start_time))
                n_batches_eval = 100
                if 'hybrid' in args.attack:
                    f_eval = robust_test_hybrid
                else:
                    f_eval = robust_test
                print('Training set validation')
                acc_train, acc_adv_train, train_loss, train_loss_adv, _ = f_eval(net, 
                    criterion, loader_train, args, attack_params, epoch, '', 
                    None, n_batches=100, train_data=True, training_time=True)
            #     if args.save_checkpoint:
                ckpt_path = 'checkpoint_' + str(0)
                torch.save(net.state_dict(), model_dir_name + ckpt_path)
#                     if train_loss_adv<best_loss_adv and epoch>0:
#                         ckpt_path_best = 'checkpoint_' + str(epoch)
#                         torch.save(net.state_dict(), model_dir_name + ckpt_path_best)
#                         best_loss_adv = train_loss_adv
                print('Train loss - Adv: %s Ben: %s' %
                    (train_loss_adv, train_loss))
                scheduler.step()
            print('Test set validation')
            # Running validation
            acc_test, acc_adv_test, test_loss, test_loss_adv, _ = f_eval(net, 
                criterion, loader_test, args, attack_params, epoch, '', 
                None,n_batches=n_batches_eval, train_data=False, training_time=True) 
            emp_loss.append([train_loss_adv, train_loss])
        emp_losses.append(emp_loss)
    emp_losses_all.append(emp_losses)

[]
2 500
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #1 epoch = 0.690
Training set validation
here
Clean accuracy: 99.10% (991/1000)
Adversarial accuracy: 83.60% (836/1000)
Train loss - Adv: 0.51951844 Ben: 0.027568216
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #2 epoch = 0.516
Training set validation
here
Clean accuracy: 98.90% (989/1000)
Adversarial accuracy: 83.10% (831/1000)
Train loss - Adv: 0.44710627 Ben: 0.031131139
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #3 epoch = 0.672
Training set validation
here
Clean accuracy: 98.30% (983/1000)
Adversarial accuracy: 82.90% (829/1000)
Train loss - Adv: 0.44868168 Ben: 0.050268255
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #4 epoch = 0.747
Training set validation
here
Clean accuracy: 98.80% (988/1000)
Adversari

time_taken for #32 epoch = 0.699
Training set validation
here
Clean accuracy: 98.90% (989/1000)
Adversarial accuracy: 83.40% (834/1000)
Train loss - Adv: 0.3967238 Ben: 0.03161827
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #33 epoch = 0.689
Training set validation
here
Clean accuracy: 98.60% (986/1000)
Adversarial accuracy: 84.00% (840/1000)
Train loss - Adv: 0.39146817 Ben: 0.037558515
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #34 epoch = 0.694
Training set validation
here
Clean accuracy: 98.90% (989/1000)
Adversarial accuracy: 83.30% (833/1000)
Train loss - Adv: 0.39824677 Ben: 0.033152483
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #35 epoch = 0.624
Training set validation
here
Clean accuracy: 98.90% (989/1000)
Adversarial accuracy: 83.70% (837/1000)
Train loss - Adv: 0.39245015 Ben: 0.03353242
Current learning 

here
Clean accuracy: 98.55% (1971/2000)
Adversarial accuracy: 82.70% (1654/2000)
Train loss - Adv: 0.41478366 Ben: 0.058541596
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #13 epoch = 1.252
Training set validation
here
Clean accuracy: 98.10% (1962/2000)
Adversarial accuracy: 82.50% (1650/2000)
Train loss - Adv: 0.41815335 Ben: 0.11871097
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #14 epoch = 1.307
Training set validation
here
Clean accuracy: 98.10% (1962/2000)
Adversarial accuracy: 82.30% (1646/2000)
Train loss - Adv: 0.46765167 Ben: 0.07757427
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #15 epoch = 1.339
Training set validation
here
Clean accuracy: 98.45% (1969/2000)
Adversarial accuracy: 82.40% (1648/2000)
Train loss - Adv: 0.41238037 Ben: 0.067463934
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta

time_taken for #43 epoch = 1.547
Training set validation
here
Clean accuracy: 98.20% (1964/2000)
Adversarial accuracy: 82.55% (1651/2000)
Train loss - Adv: 0.41256413 Ben: 0.09859475
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #44 epoch = 1.563
Training set validation
here
Clean accuracy: 98.50% (1970/2000)
Adversarial accuracy: 82.65% (1653/2000)
Train loss - Adv: 0.40837258 Ben: 0.07488471
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #45 epoch = 1.584
Training set validation
here
Clean accuracy: 97.95% (1959/2000)
Adversarial accuracy: 82.50% (1650/2000)
Train loss - Adv: 0.42030516 Ben: 0.09992154
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #46 epoch = 1.557
Training set validation
here
Clean accuracy: 98.65% (1973/2000)
Adversarial accuracy: 82.75% (1655/2000)
Train loss - Adv: 0.4090201 Ben: 0.05485792
Current lea

here
Clean accuracy: 98.92% (4946/5000)
Adversarial accuracy: 83.16% (4158/5000)
Train loss - Adv: 0.40198615 Ben: 0.03648602
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #24 epoch = 6.918
Training set validation
here
Clean accuracy: 98.84% (4942/5000)
Adversarial accuracy: 82.82% (4141/5000)
Train loss - Adv: 0.42279586 Ben: 0.05715755
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #25 epoch = 7.359
Training set validation
here
Clean accuracy: 98.80% (4940/5000)
Adversarial accuracy: 83.74% (4187/5000)
Train loss - Adv: 0.40520504 Ben: 0.07021128
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0.3011748492717743
time_taken for #26 epoch = 7.669
Training set validation
here
Clean accuracy: 98.52% (4926/5000)
Adversarial accuracy: 82.88% (4144/5000)
Train loss - Adv: 0.4035612 Ben: 0.06070687
Current learning rate: 0.1
Current eps: 1.2046993970870972, delta: 0

...

time_taken for #19 epoch = 0.782
Training set validation
here
Clean accuracy: 100.00% (1000/1000)
Adversarial accuracy: 100.00% (1000/1000)
Train loss - Adv: 0.0007107592 Ben: 7.7894e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #20 epoch = 0.731
Training set validation
here
Clean accuracy: 100.00% (1000/1000)
Adversarial accuracy: 100.00% (1000/1000)
Train loss - Adv: 0.00067713927 Ben: 7.55849e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #21 epoch = 0.797
Training set validation
here
Clean accuracy: 100.00% (1000/1000)
Adversarial accuracy: 100.00% (1000/1000)
Train loss - Adv: 0.00064958236 Ben: 7.259842e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #22 epoch = 0.791
Training set validation
here
Clean accuracy: 100.00% (1000/1000)
Adversarial accuracy: 100.00% (1000/1000)
Train loss - Adv: 0.0006291743 Be

time_taken for #49 epoch = 0.771
Training set validation
here
Clean accuracy: 100.00% (1000/1000)
Adversarial accuracy: 100.00% (1000/1000)
Train loss - Adv: 0.0005367728 Ben: 7.3665215e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #50 epoch = 0.802
Training set validation
here
Clean accuracy: 100.00% (1000/1000)
Adversarial accuracy: 100.00% (1000/1000)
Train loss - Adv: 0.0005337527 Ben: 7.817112e-05
Test set validation
here
Clean accuracy: 97.88% (4894/5000)
Adversarial accuracy: 89.46% (4473/5000)
100 1000
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #1 epoch = 0.921
Training set validation
here
Clean accuracy: 92.15% (1843/2000)
Adversarial accuracy: 86.35% (1727/2000)
Train loss - Adv: 0.38123006 Ben: 0.26321495
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #2 epoch = 0.837
Training set validation
here
Clean a

time_taken for #29 epoch = 1.550
Training set validation
here
Clean accuracy: 100.00% (2000/2000)
Adversarial accuracy: 100.00% (2000/2000)
Train loss - Adv: 0.0005574221 Ben: 3.437505e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #30 epoch = 1.551
Training set validation
here
Clean accuracy: 100.00% (2000/2000)
Adversarial accuracy: 100.00% (2000/2000)
Train loss - Adv: 0.00056168146 Ben: 3.517721e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #31 epoch = 1.604
Training set validation
here
Clean accuracy: 100.00% (2000/2000)
Adversarial accuracy: 100.00% (2000/2000)
Train loss - Adv: 0.0005628637 Ben: 3.563591e-05
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #32 epoch = 1.616
Training set validation
here
Clean accuracy: 100.00% (2000/2000)
Adversarial accuracy: 100.00% (2000/2000)
Train loss - Adv: 0.000568543 B

time_taken for #9 epoch = 7.228
Training set validation
here
Clean accuracy: 99.68% (4984/5000)
Adversarial accuracy: 94.56% (4728/5000)
Train loss - Adv: 0.13632074 Ben: 0.011808267
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #10 epoch = 7.338
Training set validation
here
Clean accuracy: 99.70% (4985/5000)
Adversarial accuracy: 88.82% (4441/5000)
Train loss - Adv: 0.270728 Ben: 0.008888356
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #11 epoch = 7.187
Training set validation
here
Clean accuracy: 99.76% (4988/5000)
Adversarial accuracy: 96.16% (4808/5000)
Train loss - Adv: 0.099829204 Ben: 0.009685011
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #12 epoch = 6.963
Training set validation
here
Clean accuracy: 99.58% (4979/5000)
Adversarial accuracy: 95.88% (4794/5000)
Train loss - Adv: 0.10855803 Ben: 0.012149942
Curre

here
Clean accuracy: 99.48% (4974/5000)
Adversarial accuracy: 97.94% (4897/5000)
Train loss - Adv: 0.056610156 Ben: 0.015470633
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #40 epoch = 6.296
Training set validation
here
Clean accuracy: 99.96% (4998/5000)
Adversarial accuracy: 97.78% (4889/5000)
Train loss - Adv: 0.061970845 Ben: 0.0021237882
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #41 epoch = 7.415
Training set validation
here
Clean accuracy: 99.88% (4994/5000)
Adversarial accuracy: 99.02% (4951/5000)
Train loss - Adv: 0.03073522 Ben: 0.0036608004
Current learning rate: 0.1
Current eps: 1.4906184673309326, delta: 0.37265461683273315
time_taken for #42 epoch = 7.086
Training set validation
here
Clean accuracy: 99.96% (4998/5000)
Adversarial accuracy: 99.38% (4969/5000)
Train loss - Adv: 0.02530272 Ben: 0.0014442542
Current learning rate: 0.1
Current eps: 1.4906184673309

In [37]:
np.save('test_output/Gaussian/robust_train_gamma%s_iter%s_restart%s' % (2.5,10,1),np.array(emp_losses_all))

### Generate test data and re-run

In [ ]:
PIK2 = "gauss_params.dat"
with open(PIK2, "rb") as f:
    param_list=pickle.load(f)

In [38]:
param_list

[[array([1.46513622, 1.4108906 ]), array([0.79329861, 0.73564343])],
 [array([0.55865223, 0.36874483, 0.37322666, 0.56447993, 0.62990381,
         0.5443139 , 0.67179704, 0.47993963, 0.11854291, 0.30300642]),
  array([0.5766785 , 0.25124798, 0.25739256, 0.58877275, 0.73316066,
         0.54745639, 0.83392474, 0.42562184, 0.02596581, 0.16965019])],
 [array([0.28163172, 0.31675939, 0.28205175, 0.03089565, 0.29226141,
         0.21154266, 0.24265924, 0.19094371, 0.23571248, 0.31851141,
         0.30602889, 0.07221236, 0.26741676, 0.14787749, 0.27011062,
         0.29249487, 0.08829067, 0.2048675 , 0.32204969, 0.15352872,
         0.24622891, 0.24026293, 0.29477393, 0.10170698, 0.31764178,
         0.12114097, 0.23765746, 0.16434927, 0.32573238, 0.11389249,
         0.05963044, 0.31758038, 0.25188843, 0.21011368, 0.1490253 ,
         0.24935645, 0.02819242, 0.15883063, 0.12074007, 0.1291993 ,
         0.19752059, 0.28904536, 0.2548167 , 0.22660979, 0.25914815,
         0.06400233, 0.083439

In [39]:
num_samples_test_list=[5000,10000,50000]
X_test_list_all=[]
for num_samples_test in num_samples_test_list:
    X_test_list=[]
    for item in param_list:
        mu=item[0]
        sigma=np.diag(item[1])
        X_1_test = rng.multivariate_normal(-1.0*mu, sigma, (num_samples_test))
        X_2_test = rng.multivariate_normal(mu, sigma, (num_samples_test))
        X_test=np.vstack((X_1_test,X_2_test))
        X_test_list.append(X_test)
    X_test_list_all.append(X_test_list)

In [40]:
result_array=np.zeros((len(num_samples_test_list),len(sample_num_list)*2,len(d_list)))

In [41]:
#Test setup
attack_iter=10
gamma=2.5
restarts=1

In [42]:
emp_losses_dict={}
eps_idx_list=[1,5]
for l,num_samples_test in enumerate(num_samples_test_list):
    emp_losses_all_test=[]
    X_test_list=X_test_list_all[l]
    for k in range(2):
        emp_losses=[]
        for i,dim in enumerate(d_list):
            emp_loss=[]
            eps_idx_curr=eps_idx_list[k]
            eps=eps_opt_list[i][eps_idx_curr].astype(np.float32)
            args = parser.parse_args("--attack=PGD_l2 --epsilon={} --attack_iter={} --gamma={} --num_restarts={}".format(eps,attack_iter,gamma,restarts).split())
            attack_params = {'attack': args.attack, 'epsilon': args.epsilon, 
                             'attack_iter': args.attack_iter, 'eps_step': args.epsilon*args.gamma/args.attack_iter,
                             'targeted': args.targeted, 'clip_min': args.clip_min,
                             'clip_max': args.clip_max,'rand_init': args.rand_init, 
                             'num_restarts': args.num_restarts}
            X_test = X_test_list[i]
            Y_test=np.zeros(2*num_samples_test)
            Y_test[num_samples_test:]=1
            for j,num_samples in enumerate(sample_num_list):
                model_dir_name = 'trained_models/Gaussian/gauss_fcn_dim' + str(dim) +'_ns' + str(num_samples) + '_eps' + str(eps)
                model_dir_name += '/'
                print(dim,num_samples)
                if not os.path.exists(model_dir_name):
                    raise ValueError()

                loader_test = load_data_gauss_test(X_test,Y_test)

                delta = args.epsilon*args.gamma/args.attack_iter
                args.track_hard=False
                args.viz=False

                net = fcn_3l(2,dim)

                # net = net.double()

                net = net.cuda()

                net.eval()
                ckpt_path = 'checkpoint_' + str(0)
                net.load_state_dict(torch.load(model_dir_name + ckpt_path))

                criterion = nn.CrossEntropyLoss() 
                # criterion = nn.CrossEntropyLoss()

                if 'hybrid' in args.attack:
                    f_eval = robust_test_hybrid
                else:
                    f_eval = robust_test
                print('Test set validation')
                n_batches_eval=int(np.floor(2*num_samples_test/50))
                # Running validation
                acc_test, acc_adv_test, test_loss, test_loss_adv, _ = f_eval(net, 
                    criterion, loader_test, args, attack_params, 0, '', 
                    None,n_batches=n_batches_eval, train_data=False, training_time=False) 
                print('Test loss - Adv: %s; Ben: %s' %
                    (test_loss_adv, test_loss))
                emp_loss.append([acc_adv_test, acc_test])
    #             result_array[j*(k+1),i,0]=(100.0-acc_adv_train)/100.0
                result_array[l,j*(k+1),i]=(100.0-acc_adv_test)/100.0
            emp_losses.append(emp_loss)
        emp_losses_all_test.append(emp_losses)
    emp_losses_dict[str(num_samples_test)]=emp_losses_all_test

2 500
Test set validation
here
Clean accuracy: 98.56% (9856/10000)
Adversarial accuracy: 81.77% (8177/10000)
Test loss - Adv: 0.4312327; Ben: 0.041809674
2 1000
Test set validation
here
Clean accuracy: 98.54% (9854/10000)
Adversarial accuracy: 82.05% (8205/10000)
Test loss - Adv: 0.43461132; Ben: 0.0498075
2 5000
Test set validation
here
Clean accuracy: 98.72% (9872/10000)
Adversarial accuracy: 82.17% (8217/10000)
Test loss - Adv: 0.41616258; Ben: 0.06409222
10 500
Test set validation
here
Clean accuracy: 97.39% (9739/10000)
Adversarial accuracy: 80.10% (8010/10000)
Test loss - Adv: 0.4716756; Ben: 0.07418473
10 1000
Test set validation
here
Clean accuracy: 96.83% (9683/10000)
Adversarial accuracy: 81.88% (8188/10000)
Test loss - Adv: 0.41303816; Ben: 0.07995538
10 5000
Test set validation
here
Clean accuracy: 97.55% (9755/10000)
Adversarial accuracy: 82.93% (8293/10000)
Test loss - Adv: 0.39237332; Ben: 0.06655273
50 500
Test set validation
here
Clean accuracy: 93.96% (9396/10000)
Adv

here
Clean accuracy: 97.69% (97685/100000)
Adversarial accuracy: 83.13% (83131/100000)
Test loss - Adv: 0.3919057; Ben: 0.0640442
50 500
Test set validation
here
Clean accuracy: 93.84% (93835/100000)
Adversarial accuracy: 88.17% (88172/100000)
Test loss - Adv: 0.65648293; Ben: 0.38346314
50 1000
Test set validation
here
Clean accuracy: 95.23% (95225/100000)
Adversarial accuracy: 89.77% (89771/100000)
Test loss - Adv: 0.56297666; Ben: 0.30357686
50 5000
Test set validation
here
Clean accuracy: 97.39% (97389/100000)
Adversarial accuracy: 90.96% (90956/100000)
Test loss - Adv: 0.34216517; Ben: 0.12965293
100 500
Test set validation
here
Clean accuracy: 91.95% (91949/100000)
Adversarial accuracy: 87.32% (87323/100000)
Test loss - Adv: 0.49260426; Ben: 0.33140028
100 1000
Test set validation
here
Clean accuracy: 94.72% (94720/100000)
Adversarial accuracy: 89.64% (89637/100000)
Test loss - Adv: 0.40690055; Ben: 0.21859267
100 5000
Test set validation
here
Clean accuracy: 97.66% (97655/100000

In [43]:
np.save('test_output/Gaussian/robust_test_array_gamma%s_iter%s_restart%s' % (gamma,attack_iter,restarts),np.array(result_array))

In [44]:
emp_losses_dict

{'5000': [[[[81.77, 98.56], [82.05, 98.54], [82.17, 98.72]],
   [[80.10000000000001, 97.39], [81.88, 96.83], [82.93, 97.55]],
   [[88.36, 93.96], [89.77000000000001, 95.00999999999999], [91.14, 97.48]],
   [[86.95, 91.47999999999999], [89.47, 94.53], [89.85, 97.76]]],
  [[[60.699999999999996, 98.46000000000001],
    [61.11, 98.22],
    [61.39, 98.74000000000001]],
   [[67.7, 96.78], [68.27, 96.52], [68.99, 97.08]],
   [[85.25, 93.78999999999999], [86.69, 95.28], [87.85, 97.45]],
   [[85.8, 91.69], [87.8, 95.19999999999999], [88.75999999999999, 97.97]]]],
 '10000': [[[[82.62, 98.76], [83.125, 98.72], [83.17999999999999, 98.89]],
   [[79.875, 97.61999999999999], [81.84, 97.005], [82.99499999999999, 97.65]],
   [[88.235, 93.91000000000001],
    [90.11, 95.30499999999999],
    [90.88000000000001, 97.32]],
   [[87.21, 92.03], [89.615, 94.71000000000001], [90.28, 97.655]]],
  [[[61.86000000000001, 98.72],
    [62.425, 98.44000000000001],
    [62.370000000000005, 98.885]],
   [[67.81, 96.75],

### Eval with optimal perturbation

In [45]:
emp_losses_opt_dict={}
eps_idx_list=[1,5]
for l,num_samples_test in enumerate(num_samples_test_list):
    emp_losses_all_test=[]
    X_test_list=X_test_list_all[l]
    for k in range(2):
        emp_losses=[]
        for i,dim in enumerate(d_list):
            emp_loss=[]
            eps_idx_curr=eps_idx_list[k]
            eps=eps_opt_list[i][eps_idx_curr].astype(np.float32)
            w = w_opt_list[i][eps_idx_curr]
            X_test = X_test_list[i]
            Y_test=np.zeros(2*num_samples_test)
            Y_test[num_samples_test:]=1
            for j,num_samples in enumerate(sample_num_list):
                model_dir_name = 'trained_models/Gaussian/gauss_fcn_dim' + str(dim) +'_ns' + str(num_samples) + '_eps' + str(eps)
                model_dir_name += '/'
                print(dim,num_samples)
                if not os.path.exists(model_dir_name):
                    raise ValueError()
#                 loader_test = load_data_gauss_test(X_test,Y_test)

                net = fcn_3l(2,dim)

                # net = net.double()

                net = net.cuda()

                net.eval()
                ckpt_path = 'checkpoint_' + str(0)
                net.load_state_dict(torch.load(model_dir_name + ckpt_path))

                loss_fn = nn.CrossEntropyLoss() 
                # criterion = nn.CrossEntropyLoss()

                print('Test set validation')
                n_batches_eval=int(np.floor(2*num_samples_test/50))
                # Running validation
                num_correct, num_correct_adv, num_samples = 0, 0, 0
                losses_adv = []
                losses_ben = []

                for b_num in range(n_batches_eval):
                    x=X_test[b_num*50:(b_num+1)*50]
                    y=Y_test[b_num*50:(b_num+1)*50]

                    y_mod = np.copy(y)
                    y_mod[np.where(y_mod==0)]=-1
                    delta_vec=np.zeros((50,dim))
                    for i_num in range(50):
                        delta_vec[i_num]= y_mod[i_num]*eps*(w/np.linalg.norm(w))
                    adv_x = x-delta_vec
                    
                    x = torch.from_numpy(x).float()
                    y = torch.from_numpy(y).float()
                    adv_x = torch.from_numpy(adv_x).float()
                    
                    x = x.cuda()
                    y = y.cuda()
                    adv_x = adv_x.cuda()
                    
                    # Predictions
                    scores = net(x.cuda()) 
                    _, preds = scores.data.max(1)
                    scores_adv = net(adv_x)
                    _, preds_adv = scores_adv.data.max(1)
#                     # Losses
#                     batch_loss_adv = loss_fn(scores_adv, y)
#                     loss_adv = torch.mean(batch_loss_adv)
#                     losses_adv.append(loss_adv.data.cpu().numpy())
#                     batch_loss_ben = loss_fn(scores, y)
#                     loss_ben = torch.mean(batch_loss_ben)
#                     losses_ben.append(loss_ben.data.cpu().numpy())
                    # Correct count
                    num_correct += (preds == y).sum()
                    num_correct_adv += (preds_adv == y).sum()
#                     num_samples += len(preds)

                acc_test = float(num_correct) / (2*num_samples_test)
                acc_adv_test = float(num_correct_adv) / (2*num_samples_test)
                print('Clean accuracy: {:.2f}% ({}/{})'.format(
                    100.*acc_test,
                    num_correct,
                    2*num_samples_test,
                ))
                print('Adversarial accuracy: {:.2f}% ({}/{})'.format(
                    100.*acc_adv_test,
                    num_correct_adv,
                    2*num_samples_test,
                ))
#                 print('Test loss - Adv: %s; Ben: %s' %
#                     (test_loss_adv, test_loss))
                emp_loss.append([acc_adv_test, acc_test])
#                 emp_loss.append([acc_test])
    #             result_array[j*(k+1),i,0]=(100.0-acc_adv_train)/100.0
                result_array[l,j*(k+1),i]=(100.0-acc_adv_test)/100.0
            emp_losses.append(emp_loss)
        emp_losses_all_test.append(emp_losses)
    emp_losses_opt_dict[str(2*num_samples_test)]=emp_losses_all_test

2 500
Test set validation
Clean accuracy: 98.56% (9856/10000)
Adversarial accuracy: 81.21% (8121/10000)
2 1000
Test set validation
Clean accuracy: 98.54% (9854/10000)
Adversarial accuracy: 81.08% (8108/10000)
2 5000
Test set validation
Clean accuracy: 98.72% (9872/10000)
Adversarial accuracy: 81.76% (8176/10000)
10 500
Test set validation
Clean accuracy: 97.39% (9739/10000)
Adversarial accuracy: 71.32% (7132/10000)
10 1000
Test set validation
Clean accuracy: 96.83% (9683/10000)
Adversarial accuracy: 70.68% (7068/10000)
10 5000
Test set validation
Clean accuracy: 97.55% (9755/10000)
Adversarial accuracy: 71.40% (7140/10000)
50 500
Test set validation
Clean accuracy: 93.96% (9396/10000)
Adversarial accuracy: 68.86% (6886/10000)
50 1000
Test set validation
Clean accuracy: 95.01% (9501/10000)
Adversarial accuracy: 70.13% (7013/10000)
50 5000
Test set validation
Clean accuracy: 97.48% (9748/10000)
Adversarial accuracy: 73.10% (7310/10000)
100 500
Test set validation
Clean accuracy: 91.48% (

### Plotting

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X_1[:,45],X_1[:,48],c='blue')
ax.scatter(X_2[:,45],X_2[:,48],c='red')

In [ ]:
fig, ax = plt.subplots()
for i,item in enumerate(eps_opt_list):
    d=d_list[i]
    curr_label=str(d)+'opt'
    ax.plot(item,optimal_loss_list[i],label=curr_label)
ax.legend()
ax.grid()
plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(16, 9), gridspec_kw={'hspace':0.5})
for i in range(len(d_list)):
    row = int(i/3)
    col = int(i%3)
    for j in range(len(sample_num_list)):
        curr_label = str(sample_num_list[j])
        axs[row,col].plot(eps_opt_list[i],emp_cost_list[i][j],label=curr_label, marker='o')
        axs[row,col].set(title='d='+str(d_list[i]),xlabel='eps',ylabel='0-1 loss')
    axs[row,col].plot(eps_opt_list[i],optimal_loss_list[i],label='optimal', linewidth=2, marker='o')
    axs[row,col].legend()
fig.delaxes(axs[1,2])
fig.delaxes(axs[1,1])

In [ ]:
fig, ax = plt.subplots()
ax.plot(eps_opt, linear_robust_loss)

ax.set(xlabel='eps', ylabel='0-1 loss',
       title='Linear classifier loss')
ax.grid()

plt.show()

In [ ]:
# Data for plotting

fig, ax = plt.subplots()
ax.plot(delta_init,eps_opt)

ax.set(xlabel='delta', ylabel='eps',
       title='Optimal eps as a function of delta')
ax.grid()

plt.show()
plt.savefig('images/analysis/3_7_fmnist_2000_l2_linear_eps_vs_delta.png')

In [ ]:
A=np.vstack((eps_opt,linear_robust_loss))

In [ ]:
np.savetxt('3_7_2000_fMNIST_l2_linear.txt',A.T, fmt='%2.5f')

In [ ]:
np.linalg.norm(mu)

In [ ]:
cos_angle = np.dot(mu,w_lda)/(np.linalg.norm(mu)*np.linalg.norm(w_lda))
angle = np.arccos(cos_angle)
angle

In [ ]:
cos_angle = np.dot(mu,w_opt[1])/(np.linalg.norm(mu)*np.linalg.norm(w_opt[1]))
angle = np.arccos(cos_angle)
angle

In [ ]:
eps=3.6
kappa_init = [4,2,1,0.5,0.25]
rhs_vec=mu
w_opt_1=[]
for kappa in kappa_init:
    print(kappa)
    lhs_mat = 0.5*(1/kappa)*(kappa*sigma+2*eps*np.eye(784))
    w=np.linalg.solve(lhs_mat,rhs_vec)
    print(np.linalg.norm(w))
    if abs(kappa-np.linalg.norm(w))<0.05:
        w_opt_1.append(w)

In [ ]:
from scipy import special

count = 0
for eps in eps_opt:
    w = w_opt[count]
    first = np.dot(sigma,w)
    second = np.dot(w,first)
    alpha_curr = 0.5*(np.sqrt(second))
    print(alpha_curr)
    min_loss = 1-special.ndtr(alpha_curr)
    print('min. loss. is %s at %s' % (min_loss, eps))
    count += 1